<a href="https://colab.research.google.com/github/Kene-ene/oilandgassupplychain/blob/main/OilandGasProfitOptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
# Import Packages
! pip install --user z3-solver
!apt-get install -y -qq glpk-utils
! pip install pyomo
from pyomo.environ import* #Pyomo Optimization Modeling Packagge
import numpy as np
#python3 -m pip install --upgrade --user ortools
#from ortools.linear_solver import pywraplp
#brew install glpk

#!pip install pyomo
#!apt install glpk-utils
#!pip install glpk

In [38]:
# Refinery Problem Data
FEEDS = ['Crude #1', 'Crude #2']
PRODUCTS = ['Gasoline', 'Kerosine', 'Fuel Oil', 'Residual']

# Feed Costs
feed_costs = {
    'Crude #1': 48,
    'Crude #2': 30
}

# Processing Costs
processing_costs = {
    'Crude #1': 1.00,
    'Crude #2': 2.00
}

# Yield Data
product_yield = {
    ('Crude #1', 'Gasoline'): 0.80,
    ('Crude #1', 'Kerosine'): 0.05,
    ('Crude #1', 'Fuel Oil'): 0.10,
    ('Crude #1', 'Residual'): 0.05,
    ('Crude #2', 'Gasoline'): 0.44,
    ('Crude #2', 'Kerosine'): 0.10,
    ('Crude #2', 'Fuel Oil'): 0.36,
    ('Crude #2', 'Residual'): 0.10,
}

# Product Sales Prices
sales_price = {
    'Gasoline': 72,
    'Kerosine': 48,
    'Fuel Oil': 42,
    'Residual': 20
}

# Production Limits
max_production = {
    'Gasoline': 24000,
    'Kerosine': 2000,
    'Fuel Oil': 6000,
    'Residual': 100000
}

In [39]:
# Model Formulation
model = ConcreteModel()

# Variables
model.x = Var(FEEDS, domain = NonNegativeReals)
model.y = Var(PRODUCTS, domain = NonNegativeReals)

# Objective Function
income = sum(sales_price[p] * model.y[p] for p in PRODUCTS)
raw_materials_cost = sum(feed_costs[f] * model.x[f] for f in FEEDS)
processing_cost = sum(processing_costs[f] * model.x[f] for f in FEEDS)

profit = income - raw_materials_cost - processing_cost
model.objective = Objective(expr = profit, sense = maximize)

# Constraints
model.constraints = ConstraintList()
for p in PRODUCTS:
  model.constraints.add (model.y[p] >= 0)
  model.constraints.add (model.y[p] <= max_production[p])
for p in PRODUCTS:
  model.constraints.add(model.y[p] == sum(model.x[f] * product_yield[(f,p)] for f in FEEDS))

In [40]:
# Solve the Model using GLPK

from pyomo.environ import SolverFactory
results = SolverFactory('glpk', executable = '/usr/bin/glpsol').solve(model)
#solver = pywraplp.Solver.CreateSolver('GLOP')
#results = solver.solve(model)
results.write()
if results.solver.status:
  model.pprint()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 573517.24137931
  Upper bound: 573517.24137931
  Number of objectives: 1
  Number of constraints: 12
  Number of variables: 6
  Number of nonzeros: 20
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.00673985481262207
# ----------------------------------------------------------
#   Solution Information
# ---------------

In [41]:
# Display Solution

print('\nProfit = ', model.objective())
print('\nDecision Variables')
print('x = ', model.x['Crude #1'](), model.x['Crude #2']())


Profit =  573517.241379312

Decision Variables
x =  26206.8965517241 6896.55172413793
